In [1]:
import math
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import numpy as np
from scipy.sparse import find
from IPython.display import clear_output

#nltk.download('punkt')
#nltk.download('stopwords')

# Cosine Similarity Scheme
*add description here*

In [2]:
# creating the dataframe with original skincare products dataset

products = pd.read_csv('data/skincare_products.csv')
products.head()

,Label,Brand,Name,Price,Rank,Ingredients,Combination,Dry,Normal,Oily,Sensitive
0,Moisturizer,LA MER,Crème de la Mer,175,4.1,"Algae (Seaweed) Extract, Mineral Oil, Petrolat...",1,1,1,1,1
1,Moisturizer,SK-II,Facial Treatment Essence,179,4.1,"Galactomyces Ferment Filtrate (Pitera), Butyle...",1,1,1,1,1
2,Moisturizer,DRUNK ELEPHANT,Protini™ Polypeptide Cream,68,4.4,"Water, Dicaprylyl Carbonate, Glycerin, Ceteary...",1,1,1,1,0
3,Moisturizer,LA MER,The Moisturizing Soft Cream,175,3.8,"Algae (Seaweed) Extract, Cyclopentasiloxane, P...",1,1,1,1,1
4,Moisturizer,IT COSMETICS,Your Skin But Better™ CC+™ Cream with SPF 50+,38,4.1,"Water, Snail Secretion Filtrate, Phenyl Trimet...",1,1,1,1,1


In [3]:
def cosine_similarity(X, Y):

  # establish stop words
  X_list = word_tokenize(X)
  Y_list = word_tokenize(Y)
  sw = stopwords.words('english')
  l1 =[];l2 =[]

  # remove stop words from strings
  X_set = {w for w in X_list if not w in sw}
  Y_set = {w for w in Y_list if not w in sw}

  # form set with words from both strings
  rvector = X_set.union(Y_set)
  for w in rvector:
    if w in X_set: l1.append(1) # create a vector
    else: l1.append(0)
    if w in Y_set: l2.append(1)
    else: l2.append(0)
  c = 0

  # cosine similarity formula
  for i in range(len(rvector)):
    c+= l1[i]*l2[i]
    cosine = c / float((sum(l1)*sum(l2))**0.5)

  return cosine

In [4]:
products.head()

,Label,Brand,Name,Price,Rank,Ingredients,Combination,Dry,Normal,Oily,Sensitive
0,Moisturizer,LA MER,Crème de la Mer,175,4.1,"Algae (Seaweed) Extract, Mineral Oil, Petrolat...",1,1,1,1,1
1,Moisturizer,SK-II,Facial Treatment Essence,179,4.1,"Galactomyces Ferment Filtrate (Pitera), Butyle...",1,1,1,1,1
2,Moisturizer,DRUNK ELEPHANT,Protini™ Polypeptide Cream,68,4.4,"Water, Dicaprylyl Carbonate, Glycerin, Ceteary...",1,1,1,1,0
3,Moisturizer,LA MER,The Moisturizing Soft Cream,175,3.8,"Algae (Seaweed) Extract, Cyclopentasiloxane, P...",1,1,1,1,1
4,Moisturizer,IT COSMETICS,Your Skin But Better™ CC+™ Cream with SPF 50+,38,4.1,"Water, Snail Secretion Filtrate, Phenyl Trimet...",1,1,1,1,1


In [5]:
# testing cosine_similarity function on products in database

# comparing first and second products in the skincare products database
test = cosine_similarity(products.iloc[0, 5], products.iloc[1, 5])
print(test)

# comparing first and third products in the skincare products database
test2 = cosine_similarity(products.iloc[0, 5], products.iloc[2, 5])
print(test2)

# comparing first and fourth products in the skincare products database
test2 = cosine_similarity(products.iloc[0, 5], products.iloc[3, 5])
print(test2)

print('Out of these comparisons, products 1 and 4 are most similar with this method')

0.22555354977384037
0.1487044791289829
0.5283120976241462
Out of these comparisons, products 1 and 4 are most similar with this method


* Create a 1472*1472 matrix of all of the similarity scores, i.e., the entry in row 1 column 2 should contain the cosine similarity between product 1 and product 2 

In [6]:
'''
cosine_similarity_matrix = np.zeros((products.shape[0], products.shape[0]))

for i in range(products.shape[0]):
    print("Generating scores for product", i)
    clear_output(wait=True)
    for j in range(i+1, products.shape[0]):
        cosine_similarity_matrix[i,j] = cosine_similarity(products.iloc[i,5], products.iloc[j,5])

cosine_similarity_matrix[0:5, 0:5]
'''

'\ncosine_similarity_matrix = np.zeros((products.shape[0], products.shape[0]))\n\nfor i in range(products.shape[0]):\n    print("Generating scores for product", i)\n    clear_output(wait=True)\n    for j in range(i+1, products.shape[0]):\n        cosine_similarity_matrix[i,j] = cosine_similarity(products.iloc[i,5], products.iloc[j,5])\n\ncosine_similarity_matrix[0:5, 0:5]\n'

In [7]:
# np.save("data/cosine_similarity_matrix.npy",cosine_similarity_matrix)

In [8]:
cosine_similarity_matrix = np.load("data/cosine_similarity_matrix.npy")

for i in range(products.shape[0]):
    for j in range(i+1, products.shape[0]):
        cosine_similarity_matrix[j,i] = cosine_similarity_matrix[i,j]

np.fill_diagonal(cosine_similarity_matrix, 1)

cosine_similarity_matrix[0:3, 0:3]

array([[1.        , 0.22555355, 0.14870448],
       [0.22555355, 1.        , 0.23268946],
       [0.14870448, 0.23268946, 1.        ]])

In [9]:
def get_5_most_similar_products(df, cos_sim_matrix: np.ndarray, index: int) -> np.ndarray:
    '''
    Gets the 5 most similar products (aside from itself) of the 
    product at the given position using the weighted distance matrix. 
    Returns a list.
    weighted_similarity_matrix: the matrix of similarity scores
    index: the positional index of the product
    '''
    sorted_indices = sorted(range(len(cos_sim_matrix[index])),
                        key = lambda i: cos_sim_matrix[index][i],
                        reverse = True)
    
    most_similar_products = df.iloc[sorted_indices[1:6],2].values
    most_similar_scores = np.round(cos_sim_matrix[index, sorted_indices[1:6]], decimals=2)

    return (most_similar_products, most_similar_scores)

In [10]:
products['Cosine Similarity Products'] = \
    products.reset_index()['index'].apply(lambda x: get_5_most_similar_products(products, cosine_similarity_matrix, x)[0])

products['Cosine Similarity Scores'] = \
    products.reset_index()['index'].apply(lambda x: get_5_most_similar_products(products, cosine_similarity_matrix, x)[1])
    
products.head(3)

,Label,Brand,Name,Price,Rank,Ingredients,Combination,Dry,Normal,Oily,Sensitive,Cosine Similarity Products,Cosine Similarity Scores
0,Moisturizer,LA MER,Crème de la Mer,175,4.1,"Algae (Seaweed) Extract, Mineral Oil, Petrolat...",1,1,1,1,1,[Little Miss Miracle Limited-Edition Crème de ...,"[1.0, 1.0, 0.6, 0.57, 0.56]"
1,Moisturizer,SK-II,Facial Treatment Essence,179,4.1,"Galactomyces Ferment Filtrate (Pitera), Butyle...",1,1,1,1,1,"[Facial Treatment Essence Mini, Facial Treatme...","[1.0, 0.85, 0.68, 0.62, 0.59]"
2,Moisturizer,DRUNK ELEPHANT,Protini™ Polypeptide Cream,68,4.4,"Water, Dicaprylyl Carbonate, Glycerin, Ceteary...",1,1,1,1,0,"[Waterfall Glacier Water Cream, Shaba Complex™...","[0.42, 0.4, 0.39, 0.39, 0.38]"


Now, we merge the weighted similarity dataframe with this one.

In [11]:
full_df = pd.read_json("data/skincare_products_1.json")
full_df.head()

,Label,Brand,Name,Price,Rank,Ingredients,Combination,Dry,Normal,Oily,Sensitive,Cosing Ref No,INCI Name,Function,Weights,Weighted Similarity Products,Weighted Similarity Scores
0,Moisturizer,LA MER,Crème de la Mer,175,4.1,"[Algae (Seaweed) Extract, Mineral Oil, Petro...",1,1,1,1,1,"[54290.0, 95058.0, 79504.0, 34040.0, 34654.0, ...","[ALGAE EXTRACT, HYDROGENATED MINERAL OIL, PETR...","[FRAGRANCE, HUMECTANT, ORAL CARE, SKIN CONDITI...","[0.20002126990000002, 0.1600170159, 0.12801361...",[Little Miss Miracle Limited-Edition Crème de ...,"[1.0, 1.0, 0.55, 0.55, 0.52]"
1,Moisturizer,SK-II,Facial Treatment Essence,179,4.1,"[Galactomyces Ferment Filtrate (Pitera), Buty...",1,1,1,1,1,"[84397, 74756, 58983, 92472, 37735, 35342, 38173]","[GALACTOMYCES FERMENT FILTRATE, BUTYLENE GLYCO...","[HUMECTANT, FRAGRANCE, HUMECTANT, SKIN CONDITI...","[0.2530733224, 0.2024586579, 0.161966926400000...","[Facial Treatment Essence Mini, Facial Treatme...","[1.0, 0.99, 0.76, 0.72, 0.71]"
2,Moisturizer,DRUNK ELEPHANT,Protini™ Polypeptide Cream,68,4.4,"[Water, Dicaprylyl Carbonate, Glycerin, Cet...",1,1,1,1,0,"[92472, 55832, 34040, 75132, 55337, 38182, 583...","[WATER, DICAPRYLYL CARBONATE, GLYCERIN, CETEAR...","[ANTIPLAQUE, SKIN CONDITIONING, SOLVENT, SKIN ...","[0.2000003831, 0.1600003065, 0.1280002452, 0.1...","[C-Tango™ Multivitamin Eye Cream, After-Sun Mi...","[0.89, 0.76, 0.75, 0.74, 0.73]"
3,Moisturizer,LA MER,The Moisturizing Soft Cream,175,3.8,"[Algae (Seaweed) Extract, Cyclopentasiloxane,...",1,1,1,1,1,"[54290.0, 75413.0, 79504.0, 34067.0, 79701.0, ...","[ALGAE EXTRACT, CYCLOPENTASILOXANE, PETROLATUM...","[FRAGRANCE, HUMECTANT, ORAL CARE, SKIN CONDITI...","[0.2000000086, 0.16000000690000002, 0.12800000...","[The Moisturizing Soft Lotion, The Concentrate...","[0.76, 0.62, 0.62, 0.5700000000000001, 0.55]"
4,Moisturizer,IT COSMETICS,Your Skin But Better™ CC+™ Cream with SPF 50+,38,4.1,"[Water, Snail Secretion Filtrate, Phenyl Tri...",1,1,1,1,1,"[92472.0, 58704.0, 79701.0, 33401.0, 74756.0, ...","[WATER, SNAIL SECRETION FILTRATE, PHENYL TRIME...","[ANTIPLAQUE, SKIN CONDITIONING, SOLVENT, SKIN ...","[0.2000000035, 0.1600000028, 0.1280000023, 0.1...",[Your Skin But Better™ CC+Illumination™ Cream ...,"[0.9, 0.75, 0.72, 0.72, 0.68]"


In [15]:
full_df['Cosine Similarity Products'] = products['Cosine Similarity Products']
full_df['Cosine Similarity Scores'] = products['Cosine Similarity Scores']
full_df.iloc[:,[2,15,16,17,18]].head(3)

,Name,Weighted Similarity Products,Weighted Similarity Scores,Cosine Similarity Products,Cosine Similarity Scores
0,Crème de la Mer,[Little Miss Miracle Limited-Edition Crème de ...,"[1.0, 1.0, 0.55, 0.55, 0.52]",[Little Miss Miracle Limited-Edition Crème de ...,"[1.0, 1.0, 0.6, 0.57, 0.56]"
1,Facial Treatment Essence,"[Facial Treatment Essence Mini, Facial Treatme...","[1.0, 0.99, 0.76, 0.72, 0.71]","[Facial Treatment Essence Mini, Facial Treatme...","[1.0, 0.85, 0.68, 0.62, 0.59]"
2,Protini™ Polypeptide Cream,"[C-Tango™ Multivitamin Eye Cream, After-Sun Mi...","[0.89, 0.76, 0.75, 0.74, 0.73]","[Waterfall Glacier Water Cream, Shaba Complex™...","[0.42, 0.4, 0.39, 0.39, 0.38]"


In [16]:
full_df

,Label,Brand,Name,Price,Rank,Ingredients,Combination,Dry,Normal,Oily,Sensitive,Cosing Ref No,INCI Name,Function,Weights,Weighted Similarity Products,Weighted Similarity Scores,Cosine Similarity Products,Cosine Similarity Scores
0,Moisturizer,LA MER,Crème de la Mer,175,4.1,"[Algae (Seaweed) Extract, Mineral Oil, Petro...",1,1,1,1,1,"[54290.0, 95058.0, 79504.0, 34040.0, 34654.0, ...","[ALGAE EXTRACT, HYDROGENATED MINERAL OIL, PETR...","[FRAGRANCE, HUMECTANT, ORAL CARE, SKIN CONDITI...","[0.20002126990000002, 0.1600170159, 0.12801361...",[Little Miss Miracle Limited-Edition Crème de ...,"[1.0, 1.0, 0.55, 0.55, 0.52]",[Little Miss Miracle Limited-Edition Crème de ...,"[1.0, 1.0, 0.6, 0.57, 0.56]"
1,Moisturizer,SK-II,Facial Treatment Essence,179,4.1,"[Galactomyces Ferment Filtrate (Pitera), Buty...",1,1,1,1,1,"[84397, 74756, 58983, 92472, 37735, 35342, 38173]","[GALACTOMYCES FERMENT FILTRATE, BUTYLENE GLYCO...","[HUMECTANT, FRAGRANCE, HUMECTANT, SKIN CONDITI...","[0.2530733224, 0.2024586579, 0.161966926400000...","[Facial Treatment Essence Mini, Facial Treatme...","[1.0, 0.99, 0.76, 0.72, 0.71]","[Facial Treatment Essence Mini, Facial Treatme...","[1.0, 0.85, 0.68, 0.62, 0.59]"
2,Moisturizer,DRUNK ELEPHANT,Protini™ Polypeptide Cream,68,4.4,"[Water, Dicaprylyl Carbonate, Glycerin, Cet...",1,1,1,1,0,"[92472, 55832, 34040, 75132, 55337, 38182, 583...","[WATER, DICAPRYLYL CARBONATE, GLYCERIN, CETEAR...","[ANTIPLAQUE, SKIN CONDITIONING, SOLVENT, SKIN ...","[0.2000003831, 0.1600003065, 0.1280002452, 0.1...","[C-Tango™ Multivitamin Eye Cream, After-Sun Mi...","[0.89, 0.76, 0.75, 0.74, 0.73]","[Waterfall Glacier Water Cream, Shaba Complex™...","[0.42, 0.4, 0.39, 0.39, 0.38]"
3,Moisturizer,LA MER,The Moisturizing Soft Cream,175,3.8,"[Algae (Seaweed) Extract, Cyclopentasiloxane,...",1,1,1,1,1,"[54290.0, 75413.0, 79504.0, 34067.0, 79701.0, ...","[ALGAE EXTRACT, CYCLOPENTASILOXANE, PETROLATUM...","[FRAGRANCE, HUMECTANT, ORAL CARE, SKIN CONDITI...","[0.2000000086, 0.16000000690000002, 0.12800000...","[The Moisturizing Soft Lotion, The Concentrate...","[0.76, 0.62, 0.62, 0.5700000000000001, 0.55]","[The Moisturizing Soft Lotion, The Moisturizin...","[0.81, 0.69, 0.63, 0.6, 0.59]"
4,Moisturizer,IT COSMETICS,Your Skin But Better™ CC+™ Cream with SPF 50+,38,4.1,"[Water, Snail Secretion Filtrate, Phenyl Tri...",1,1,1,1,1,"[92472.0, 58704.0, 79701.0, 33401.0, 74756.0, ...","[WATER, SNAIL SECRETION FILTRATE, PHENYL TRIME...","[ANTIPLAQUE, SKIN CONDITIONING, SOLVENT, SKIN ...","[0.2000000035, 0.1600000028, 0.1280000023, 0.1...",[Your Skin But Better™ CC+Illumination™ Cream ...,"[0.9, 0.75, 0.72, 0.72, 0.68]",[Your Skin But Better™ CC+Illumination™ Cream ...,"[0.97, 0.86, 0.72, 0.71, 0.52]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1467,Sun protect,KORRES,Yoghurt Nourishing Fluid Veil Face Sunscreen B...,35,3.9,"[Water, Alcohol Denat., Potassium Cetyl Phos...",1,1,1,1,1,"[92472.0, 74174.0, 79512.0, 34040.0, 74860.0, ...","[WATER, ALCOHOL DENAT., POTASSIUM CETYL PHOSPH...","[ANTIPLAQUE, SKIN CONDITIONING, SOLVENT, ANTIF...","[0.200758444, 0.1606067552, 0.1284854041, 0.10...","[Ultra Facial Oil-Free Gel-Cream, Clarifying L...","[0.72, 0.71, 0.71, 0.71, 0.71]","[Pomegranate AHA & Enzymes Resurfacing Mask, H...","[0.54, 0.45, 0.45, 0.44, 0.44]"
1468,Sun protect,KATE SOMERVILLE,Daily Deflector™ Waterlight Broad Spectrum SPF...,48,3.6,"[Water, Isododecane, Dimethicone, Butylocty...",0,0,0,0,0,"[92472, 34648, 33401, 31733, 74635, 75266, 340...","[WATER, ISODODECANE, DIMETHICONE, BUTYLOCTYL S...","[ANTIPLAQUE, SKIN CONDITIONING, SOLVENT, PERFU...","[0.2000265881, 0.1600212705, 0.128017016400000...",[Super Fluid UV Mineral Defense Titanium Dioxi...,"[0.87, 0.79, 0.79, 0.78, 0.78]",[Daily Deflector™ Moisturizer Broad Spectrum S...,"[0.59, 0.47, 0.45, 0.45, 0.45]"
1469,Sun protect,VITA LIBERATA,Self Tan Dry Oil SPF 50,54,3.5,"[Water, Dihydroxyacetone, Glycerin, Scleroc...",0,0,0,0,0,"[92472, 75563, 34040, 58399, 35360, 55560, 

In [17]:
full_df.to_json("data/skincare_products_1.json")